# Möbius Plateau Auto‑Explorer Ω✦ (grand‑sweep)

Paste your fine‑scan rows into **Cell 2**, then press **Runtime ▸ Run all**.

What it does – totally hands‑off:

1. Reads your scan table **(bits, k, φ, λ, rel_err)**.
2. Tags rows into plateaux at three granularities (ε = 1e‑4, 5e‑6, 1e‑5).
3. For **every constant** listed below it applies *all* of these ops  
   (`x` is the rel_err column):

```
x⁻¹, 1‑x, √x, ∛x, x¹ᐟ⁹, x⁹, x⁻¹ᐟ⁹, x÷9, x×9
```

4. Constants swept:

| symbol | value | note |
| ------ | ----- | ---- |
| φ      | 1.618 033 988 75… | golden ratio |
| α⁻¹    | 137.035 999 084… | fine‑structure |
| δ      | 4.669 201 609…   | Feigenbaum δ |
| α_F    | 2.502 907 875…   | Feigenbaum α |
| π      | 3.141 592 654…   | pi |
| e      | 2.718 281 828…   | Euler’s number |
| γ      | 0.577 215 664 9… | Euler‑Mascheroni |
| φᵦ     | 0.010 0101…      | Fibonacci binary word (0.010010100…) |

5. Prints the first 60 rows for every (ε, op) combo so you can eyeball structure.
6. Saves two Excel files in `/content`:

* **plateaus_raw.xlsx** – untouched rows  
* **plateaus_ops.xlsx** – one sheet per ε+op

No widgets, no parameters – just run.


In [1]:

import pandas as pd, numpy as np

# ▼ Paste / update your raw scan rows here ▼
raw_rows = [
    #  bits  k     φ‑micro      λ      rel_err
    (64,  3.6, 0.003942, 0.0, 0.4117525875906678269),
    (64,  3.8, 0.003942, 0.0, 0.4117525875906678269),
    (64,  4.0, 0.003942, 0.0, 0.4117525875906678269),
    (64,  4.2, 0.003942, 0.0, 0.4117525875906678269),
    (128, 3.6, 0.003942, 0.0, 0.4117525875906678271),
    (128, 3.8, 0.003942, 0.0, 0.4117525875906678271),
    (128, 4.0, 0.003942, 0.0, 0.4117525875906678271),
    (128, 4.2, 0.003942, 0.0, 0.4117525875906678271),
]

df = pd.DataFrame(raw_rows, columns=['bits','k','phi','lam','rel_err'])
display(df.head())


,bits,k,phi,lam,rel_err
0,64,3.6,0.003942,0.0,0.411753
1,64,3.8,0.003942,0.0,0.411753
2,64,4.0,0.003942,0.0,0.411753
3,64,4.2,0.003942,0.0,0.411753
4,128,3.6,0.003942,0.0,0.411753


In [2]:

import math, itertools, numpy as np, os

# == built‑in constants ==
phi    = (1+5**0.5)/2
phi_inv = 1/phi
alpha_inv = 137.035999084   # fine‑structure 1/α
alpha = 1/alpha_inv
delta = 4.669201609        # Feigenbaum δ
alpha_F = 2.502907875      # Feigenbaum α
pi = math.pi
e  = math.e
gamma = 0.5772156649015328606

# Fibonacci binary‑word constant 0.0100101001001… (as real)
phi_b = sum( int(ch)*2**-(i+1) for i,ch in enumerate('0100101001001'*30) )

# Helper inverses / complements
def compl(x): return 1-x

consts = {
    'φ':phi, 'φ⁻¹':phi_inv, 'α':alpha, 'α⁻¹':alpha_inv,
    'δ':delta, 'α_F':alpha_F, 'π':pi, 'e':e, 'γ':gamma, 'φᵦ':phi_b,
}

# Compose lambdas
OPS = {}
def add_op(name, fn): OPS[name]=fn

# multiply / divide by constants; we treat str for numbers too
for name,cval in consts.items():
    OPS[f'×{name}'] = lambda x,c=cval: x*c
    OPS[f'÷{name}'] = lambda x,c=cval: x/c
    OPS[f'{name}×'] = lambda x,c=cval: c*x   # constant standalone sanity

# base lambdas for numeric ops
OPS.update({
    '×9' : lambda x: x*9,
    '÷9' : lambda x: x/9,
    'x⁻¹': lambda x: np.where(x!=0, 1/x, np.inf),
    '1‑x': lambda x: compl(x),
    '√x':  lambda x: np.sqrt(np.abs(x)),
    '∛x':  lambda x: np.cbrt(x),
    'x¹ᐟ⁹': lambda x: np.sign(x)*np.abs(x)**(1/9),
    'x⁹'  : lambda x: np.power(x,9),
    'x⁻¹ᐟ⁹': lambda x: np.sign(x)*np.abs(x)**(-1/9),
})

print(f"Loaded {len(OPS)} ops")


Loaded 39 ops


In [3]:

import pandas as pd, numpy as np, math, os

def tag_plateaus(frame: pd.DataFrame, eps: float):
    base = frame['rel_err'].min()
    out  = frame.copy()
    out['plateau'] = ((out['rel_err']-base)/eps).astype(int)
    return out

views = []
for eps in [1e-4,5e-6,1e-5]:
    tagged = tag_plateaus(df, eps)
    for op_name, op_fun in OPS.items():
        tmp = tagged.copy()
        tmp['ε'] = eps
        tmp['op'] = op_name
        tmp['rel_err'] = op_fun(tmp['rel_err'])
        views.append(tmp.iloc[:60])  # keep first 60 rows per view

out = pd.concat(views, ignore_index=True)

# Save
raw_file = 'plateaus_raw.xlsx'
ops_file = 'plateaus_ops.xlsx'
df.to_excel(raw_file, index=False)
out.to_excel(ops_file, index=False)
print(f"Wrote {raw_file} and {ops_file}")
display(out.head(30))


Wrote plateaus_raw.xlsx and plateaus_ops.xlsx


,bits,k,phi,lam,rel_err,plateau,ε,op
0,64,3.6,0.003942,0.0,0.666230,0,0.0001,×φ
1,64,3.8,0.003942,0.0,0.666230,0,0.0001,×φ
2,64,4.0,0.003942,0.0,0.666230,0,0.0001,×φ
3,64,4.2,0.003942,0.0,0.666230,0,0.0001,×φ
4,128,3.6,0.003942,0.0,0.666230,0,0.0001,×φ
5,128,3.8,0.003942,0.0,0.666230,0,0.0001,×φ
6,128,4.0,0.003942,0.0,0.666230,0,0.0001,×φ
7,128,4.2,0.003942,0.0,0.666230,0,0.0001,×φ
8,64,3.6,0.003942,0.0,0.254477,0,0.0001,÷φ
9,64,3.8,0.003942,0.0,0.254477,0,0.0001,÷φ
